In [50]:
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import pickle


In [51]:
def resize_image(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return cv2.resize(gray , (128 , 64))

In [52]:
def extract_hand_segment(image):
    # Convert the input BGR image to the YCrCb color space
    ycrcb_image = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

    # Split the channels of the YCrCb image into Y, Cr, and Cb channels
    y_channel, cr_channel, cb_channel = cv2.split(ycrcb_image)

    # Thresholding to create a binary mask for the hand region based on Cr and Cb channels
    _, cr_mask = cv2.threshold(cr_channel, 133, 173, cv2.THRESH_BINARY)
    _, cb_mask = cv2.threshold(cb_channel, 77, 127, cv2.THRESH_BINARY)
    hand_mask = cv2.bitwise_and(cr_mask, cb_mask)

    # Dilate the hand mask to fill in any gaps
    kernel = np.ones((5, 5), np.uint8)
    dilated_mask = cv2.dilate(hand_mask, kernel, iterations=7)

    # Apply a closing operation to fill any black gaps in the hand
    closed_mask = cv2.morphologyEx(dilated_mask, cv2.MORPH_CLOSE, kernel)

    # Find the contours of the object in the mask
    _, contours, hierarchy = cv2.findContours(closed_mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Get the largest contour (which should be the hand)
    contour_sizes = [(cv2.contourArea(contour), contour) for contour in contours]
    largest_contour = max(contour_sizes, key=lambda x: x[0])[1]

    # Create a mask of the hand contour
    hand_mask = np.zeros_like(dilated_mask)
    cv2.drawContours(hand_mask, [largest_contour], 0, 255, cv2.FILLED)

    # Apply erosion to reduce the size of the segmented area
    kernel = np.ones((3, 3), np.uint8)
    eroded_mask = cv2.erode(hand_mask, kernel, iterations=12)

    # Apply the hand mask to the original image to extract the hand
    hand_segment = cv2.bitwise_and(image, image, mask=eroded_mask)

    # Return the segmented hand image
    return hand_segment

In [53]:
# Define the image transformation functions
def flip(image):
    # Flip the image horizontally
    return cv2.flip(image, 1)

def rotate(image):
    # Rotate the image by 30 degrees
    rows, cols = image.shape[:2]
    M = cv2.getRotationMatrix2D((cols/2, rows/2), 30, 1)
    return cv2.warpAffine(image, M, (cols, rows))

In [54]:
def preProcessing(img):
    
    # Segment the hand
    img = extract_hand_segment(img)

    # Resize the image
    img = resize_image(img)

    return img


In [55]:
def augmentation(img):

    # Rotate the image
    rotated = rotate(img)

    # Flip the rotated image
    flipped = flip(rotated)

    return flipped

In [56]:
# Prepare the training data
X = []
y = []
images = []
for i in range(0, 6):
    for j in range(1, 180):
        filename = 'men/{}/{}_men ({}).JPG'.format(i, i , j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        
        
        # Preprocess the image
        finalImg = preProcessing(img)

        # Save the final image to array of images
        images.append(finalImg)

        # Save the label of the image
        y.append(i)
        

        # Make new image of the preprocessed image 
        augmentedImage = augmentation(finalImg)

        # Save the new image to array of images
        images.append(augmentedImage)

        # Save the label of the image
        y.append(i)


        filename = 'Woman/{}/{}_woman ({}).JPG'.format(i, i, j)
        img = cv2.imread(filename)
        if  img is None:
            continue    
        

        # Preprocess the image
        finalImg = preProcessing(img)

        # Save the final image to array of images
        images.append(finalImg)

        # Save the label of the image
        y.append(i)
        

        # Make new image of the preprocessed image 
        augmentedImage = augmentation(finalImg)

        # Save the new image to array of images
        images.append(augmentedImage)

        # Save the label of the image
        y.append(i)

        


In [57]:
# Extract hog features from the preprocessed images and save it.
hog_features = hog(images[0], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
X = np.zeros((len(y) , len(hog_features)))
for i in range(len(y)):
     hog_features = hog(images[i], orientations=12, pixels_per_cell=(12, 12), cells_per_block=(3, 3), block_norm='L2-Hys')
     X[i] = (np.array(hog_features))   

In [58]:
# Split the data to training data and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
# Try different svm kernels to find the best one for this application.
kernel= ['linear', 'poly', 'rbf', 'sigmoid']
for i in kernel:
    # Train the SVM model
    model = svm.SVC(kernel=i, C=100, gamma=0.1, degree=6)
    model.fit(X_train, y_train)
    accuracy = model.score(X_test, y_test)
    print(f'Accuracy of {i} kernel: {accuracy*100}%')

Accuracy of linear kernel: 69.0909090909091%
Accuracy of poly kernel: 82.51748251748252%
Accuracy of rbf kernel: 83.63636363636363%
Accuracy of sigmoid kernel: 24.055944055944057%


In [60]:
# Fit the data into the best model.
model = svm.SVC(kernel='rbf', C=100, gamma=0.1, degree=6)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
print(f'Accuracy of rbf kernel: {accuracy*100}%')

Accuracy of rbf kernel: 83.63636363636363%


In [61]:
# Save the model to a file.
filename = 'model.sav'
pickle.dump(model, open(filename, 'wb'))